# Evolver Loop 3 - LB Feedback Analysis

## Situation Summary
- **Best LB Score**: 70.6224 (from exp_001 and exp_002)
- **Target**: 68.8877
- **Gap**: 1.73 points (2.5%)

## Key Findings from Experiments
1. exp_000: Baseline from snapshot - had overlaps, rejected by Kaggle
2. exp_001: Fixed overlaps - LB 70.6224 (passed)
3. exp_002: Python optimization (N=1 rotation, fractional translation, backward propagation) - LB 70.6224 (no improvement)

## Critical Insight
The baseline is already at a LOCAL OPTIMUM. Simple local search cannot improve it.
We need FUNDAMENTALLY DIFFERENT approaches.

In [ ]:
# Analyze the score breakdown to understand where improvements are possible
import numpy as np

# Score breakdown from baseline (from session_state)
# N=1: 0.6612, N=2-5: 1.7189, N=6-10: 1.9490, N=11-50: 14.7036, N=51-100: 17.6063, N=101-200: 33.9768

score_breakdown = {
    'N=1': 0.6612,
    'N=2-5': 1.7189,
    'N=6-10': 1.9490,
    'N=11-50': 14.7036,
    'N=51-100': 17.6063,
    'N=101-200': 33.9768
}

total = sum(score_breakdown.values())
print(f"Total baseline score: {total:.4f}")
print(f"Target: 68.8877")
print(f"Gap: {total - 68.8877:.4f}")
print()
print("Score contribution by N range:")
for k, v in score_breakdown.items():
    pct = v / total * 100
    print(f"  {k}: {v:.4f} ({pct:.1f}%)")

In [ ]:
# Calculate theoretical minimum for N=1
# Tree dimensions: height 1.0, width 0.7
# At 45 degrees, the bounding box is minimized

import math

# Tree vertices
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def rotate_point(x, y, angle_deg):
    angle_rad = math.radians(angle_deg)
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    return x * cos_a - y * sin_a, x * sin_a + y * cos_a

def get_bbox_for_angle(angle):
    rotated = [rotate_point(x, y, angle) for x, y in zip(TX, TY)]
    xs = [p[0] for p in rotated]
    ys = [p[1] for p in rotated]
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    return max(width, height)

# Check angles around 45 degrees
print("Bounding box side for different angles:")
for angle in [0, 30, 45, 60, 90]:
    side = get_bbox_for_angle(angle)
    score = side ** 2
    print(f"  {angle}°: side = {side:.6f}, score = {score:.6f}")

# Find optimal angle
best_angle = 0
best_side = float('inf')
for angle in range(0, 360):
    side = get_bbox_for_angle(angle)
    if side < best_side:
        best_side = side
        best_angle = angle

print(f"\nOptimal angle: {best_angle}° with side {best_side:.6f}")
print(f"N=1 optimal score: {best_side**2:.6f}")

## Strategic Analysis

### Why Local Search Failed
1. The baseline was already optimized by sophisticated C++ tools (bbox3, SA)
2. Simple moves (translation, rotation) cannot escape the local optimum
3. The configurations are already tightly packed

### What Could Work

1. **Simulated Annealing from Scratch** - Accept worse moves to escape local optima
2. **Random Restart** - Start from different initial configurations
3. **Constructive Heuristics** - Build solutions from scratch (like zaburo kernel)
4. **Ensemble Best Per-N** - Combine best solutions from multiple sources

### Key Insight from Zaburo Kernel
The zaburo kernel shows a CONSTRUCTIVE approach:
- Build solutions using alternating rows (0° and 180° angles)
- Trees at 0° and 180° can interlock efficiently
- This achieves 88.33 initially but provides a DIFFERENT starting point

### Recommended Next Steps
1. Implement simulated annealing FROM SCRATCH in Python
2. Start from RANDOM configurations, not the optimized baseline
3. Focus on finding DIFFERENT local optima, not improving the current one

In [ ]:
# Calculate how much improvement we need per N range

target = 68.8877
current = 70.6158
gap = current - target

print(f"Total gap to close: {gap:.4f} points")
print()
print("If we could improve each N range by the same percentage:")
required_pct = gap / current * 100
print(f"  Required improvement: {required_pct:.2f}%")
print()
print("Improvement needed per range (proportional):")
for k, v in score_breakdown.items():
    needed = v * (gap / current)
    print(f"  {k}: {needed:.4f} points")

## Conclusion

The experiments confirm that:
1. N=1 is already optimal at 45°
2. Local search cannot improve the baseline
3. We need GLOBAL optimization or DIFFERENT initial configurations

**NEXT EXPERIMENT MUST:**
1. Implement simulated annealing from scratch (not using binaries)
2. Start from random/constructive configurations
3. Allow accepting worse moves to escape local optima
4. Focus on N values with most room for improvement (large N)